#### Import classes

In [ ]:
from impl_to_play import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, FullQueryEngine
# from impl import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, BasicQueryEngine
# from impl_with_modification import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, BasicQueryEngine

path = 'http://10.201.42.7:9999/blazegraph/'
grp_endpoint = f'{path}sparql'
# java -server -Xmx1g -jar blazegraph.jar

# set Category methods

real_path = 'trial.db'
cat = CategoryUploadHandler()
cat.setDbPathOrUrl(real_path)
# cat.pushDataToDb('test_data/scimago.json')

cat_qh = CategoryQueryHandler()
cat_qh.setDbPathOrUrl(real_path)

# set Journal methods

jou = JournalUploadHandler()
jou.setDbPathOrUrl(grp_endpoint)
# jou.pushDataToDb('test_data/doaj.csv')

jou_qh = JournalQueryHandler()
jou_qh.setDbPathOrUrl(grp_endpoint)

# set Engine methods

que = FullQueryEngine()
que.addJournalHandler(jou_qh)
que.addCategoryHandler(cat_qh)

#### Check JournalQueryHandler

In [2]:
# result_j1 = jou_qh.getById('2532-2615') # 2532-2615 2385-2615
result_j1 = jou_qh.getAllJournals()
# result_j1 = jou_qh.getJournalsWithTitle('Cuadernos de Lingüística de El Colegio')
# result_j1 = jou_qh.getJournalsPublishedBy('ITB Journal Publisher')
# result_j1 = jou_qh.getJournalsWithLicense('CC BY')
# result_j1 = jou_qh.getJournalsWithAPC()
# result_j1 = jou_qh.getJournalsWithDOAJSeal()

result_j1

,journal,title,identifier,languages,publisher,license,apc,seal
0,https://comp-data.github.io/resjournal-26,Revista Ciencias Veterinarias,2215-4507,Spanish,"Universidad Nacional, Costa Rica",CC BY-NC-ND,No,No
1,https://comp-data.github.io/resjournal-26,Revista Ciencias Veterinarias,2215-4507,English,"Universidad Nacional, Costa Rica",CC BY-NC-ND,No,No
2,https://comp-data.github.io/resjournal-135,Journal of Techno-Social,2229-8940,English,Penerbit UTHM,Publisher's own license,No,No
3,https://comp-data.github.io/resjournal-135,Journal of Techno-Social,2229-8940,Malay (macrolanguage),Penerbit UTHM,Publisher's own license,No,No
4,https://comp-data.github.io/resjournal-70,Journal of Chromatography Open,2772-3917,English,Elsevier,"CC BY, CC BY-NC-ND",Yes,No
...,...,...,...,...,...,...,...,...
546,https://comp-data.github.io/resjournal-30,Półrocznik Językoznawczy Tertium,2543-7844,Polish,Cracow Tertium Society for the Promotion of La...,CC BY-NC-ND,No,No
547,https://comp-data.github.io/resjournal-85,Инженерные технологии и системы,2658-6525,Russian,National Research Mordova State University; MRSU,CC BY,No,No
548,https://comp-data.github.io/resjournal-85,Инженерные технологии и системы,2658-6525,English,National Research Mordova State University; MRSU,CC BY,No,No
549,https://comp-data.github.io/resjournal-85,Инженерные технологии и системы,2658-4123,Russian,National Research Mordova State University; MRSU,CC BY,No,No


#### Check CategoryQueryHandler

In [5]:
# result_c1 = cat_qh.getAllCategories()
# result_c1 = cat_qh.getAllAreas()
result_c1 = cat_qh.getCategoriesWithQuartile({'Q1'})
# result_c1 = cat_qh.getCategoriesAssignedToAreas({'Medicine'})  
# result_c1 = cat_qh.getAreasAssignedToCategories({'Artificial Intelligence', 'Oncology'}) 

# print(result_c1.loc[result_c1['category_id']=='Agronomy and Crop Science'])
result_c1

,category_id,category_quartile
0,Hematology,Q1
1,Oncology,Q1
2,Artificial Intelligence,Q1
3,Human-Computer Interaction,Q1
4,Software,Q1
...,...,...
303,Review and Exam Preparation,Q1
304,Drug Guides,Q1
305,Pathophysiology,Q1
306,Nurse Assisting,Q1


### Check QueryEngine

In [2]:
# BasicQueryEngine

# result_q1 = que.getById('2532-2615') # 2532-2615 2385-2615
result_q1 = que.getAllJournals()
# result_q1 = que.getJournalsWithTitle('Cuadernos de Lingüística de El Colegio')
# result_q1 = que.getJournalsPublishedBy('ITB Journal Publisher')
# result_q1 = que.getJournalsWithLicense('CC BY')
# result_q1 = que.getJournalsWithAPC()
# result_q1 = que.getJournalsWithDOAJSeal()
# result_q1 = que.getAllCategories()
# result_q1 = que.getAllAreas()
# result_q1 = que.getCategoriesWithQuartile({'Q1'})
# result_q1 = que.getCategoriesAssignedToAreas({'Medicine'})
# result_q1 = que.getAreasAssignedToCategories({'Artificial Intelligence', 'Oncology'})
result_q1



identifier: ...  1983-9979
issn:  1983-9979 eissn:  None
identifier: ...  2224-9281, 2414-990X
issn:  2224-9281 eissn:  2414-990X
identifier: ...  1410-7791, 2528-0805
issn:  1410-7791 eissn:  2528-0805
identifier: ...  2516-3957, 2516-3949
issn:  2516-3957 eissn:  2516-3949
identifier: ...  2715-7989, 2686-5343
issn:  2715-7989 eissn:  2686-5343
identifier: ...  2542-1956, 1994-2540
issn:  2542-1956 eissn:  1994-2540
identifier: ...  2541-8971, 1816-9775
issn:  2541-8971 eissn:  1816-9775
identifier: ...  2162-6901, 0887-2058
issn:  2162-6901 eissn:  0887-2058
identifier: ...  2220-802X
issn:  2220-802X eissn:  None
hasArea:  ['area_3', 'area_14']
identifier: ...  2764-2666
issn:  2764-2666 eissn:  None
identifier: ...  2782-3229, 0321-4796
issn:  2782-3229 eissn:  0321-4796
identifier: ...  1824-5226
issn:  1824-5226 eissn:  None
identifier: ...  2656-7806
issn:  2656-7806 eissn:  None
identifier: ...  2243-4712
issn:  2243-4712 eissn:  None
hasArea:  ['area_14']
identifier: ...  228

In [ ]:
# FullQueryEngine

result_q1 = que.JournalsInCategoryWithQuartile()
result_q1 = que.JournalsInAreasWithLicense()
result_q1 = que.DiamondJournalsInAreasAndCategoriesWithQuartile()

result_q1

#### edo stuff

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from sqlite3 import connect
import SPARQLWrapper
from rdflib import Graph, URIRef, Literal, RDF 
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [ ]:
df = pd.read_csv('test_data/doaj.csv')
counter = 0
# df.info()
entrambi_nan_1 = df[['Journal ISSN (print version)', 'Journal EISSN (online version)']].isna().all(axis=1).sum()
# print(entrambi_nan_1)

entrambi_non_nan = ~(df['Journal ISSN (print version)'].isna()) | ~(df['Journal EISSN (online version)'].isna())

# Conta il numero di volte in cui la condizione è True
conteggio = entrambi_non_nan.sum()
# print(conteggio)

seal_count = (df['Journal title'] == 'Revista Ciencias Marinas y Costeras') # Revista Ciencias Marinas y Costeras
df[df['Journal title'] == 'Revista Ciencias Marinas y Costeras']

,Journal title,Journal ISSN (print version),Journal EISSN (online version),Languages in which the journal accepts manuscripts,Publisher,DOAJ Seal,Journal license,APC
176,Revista Ciencias Marinas y Costeras,1659-455X,1659-407X,"English, Spanish","Universidad Nacional, Costa Rica",No,CC BY-NC-SA,No


In [ ]:
    def getCategoriesWithQuartile(self, quartiles: set):
        with connect(self.dbPathOrUrl) as con:
            if not quartiles:
                # Se l'insieme dei quartili in input è vuoto, seleziona tutte le categorie
                query = """
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                """
                df = pd.read_sql_query(query, con)
            else:
                # Altrimenti, seleziona solo le categorie con i quartili specificati
                placeholders = ', '.join(['?'] * len(quartiles))
                query = f"""
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                    WHERE quartile IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(quartiles))
        return df

    # Prendere tutte le categorie associate a una lista di aree
    def getCategoriesAssignedToAreas(self, areas: set):
        with connect(self.dbPathOrUrl) as con:
            if not areas:
                query = f"""
                    SELECT DISTINCT c.id, a.area
                    FROM categories c 
                """
                df = pd.read_sql_query(query, con)
            else:
                placeholders = ', '.join(['?'] * len(areas))
                query = f"""
                SELECT DISTINCT c.id, a.area
                FROM categories c
                JOIN areas a ON c.item_internal_id = a.item_internal_id
                WHERE a.area IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(areas))
        return df

## Cose da vedere con Chiara

- JournalUploadHandler
    - siamo sicuri di dover splittare 'language'?
    - penso si debbano aggiungere delle colonne che traccino hasCategory e hasArea nel db a grafo, mostrale nei test le tabelle che ti sono genute con i metodi e poi vai in BasicMashup a createObjectList (per le parti 0..* creai prima una lista vuota ed if row[hasCategory] != "" crei una nuova lista con i valori interni)

- CI serve una chiave primaria, un id univoco in ogni tabella 

In [ ]:
import json
path = 'test_data/scimago.json'
with open(path, 'r') as a:
    file = json.load(a)

counter = list()
for items in file:
    for id in items.get('identifiers'):
        # print(id)
        # break 
        if id == '1474-175X':
            print(items)
    # counter.append(items.get('identifiers'))
# print(len(counter))
    

{'identifiers': ['1474-175X', '1474-1768'], 'categories': [{'id': 'Cancer Research', 'quartile': 'Q1'}, {'id': 'Oncology', 'quartile': 'Q1'}], 'areas': ['Biochemistry, Genetics and Molecular Biology', 'Medicine']}


In [ ]:
import pandas as pd

df = pd.read_csv('test_data/doaj.csv')


,Journal title,Journal ISSN (print version),Journal EISSN (online version),Languages in which the journal accepts manuscripts,Publisher,DOAJ Seal,Journal license,APC
0,Prolíngua,NaN,1983-9979,Portuguese,Universidade Federal da Paraíba,No,CC BY-NC-SA,No
1,Проблеми Законності,2224-9281,2414-990X,"Ukrainian, Russian, English",Yaroslav Mudryi National Law University,No,CC BY,Yes
2,Enlightening Tourism: A Pathmaking Journal,NaN,2174-548X,English,University of Huelva,No,CC BY-NC,No
3,Scientific Journals of the Maritime University...,1733-8670,2392-0378,English,MUS,No,CC BY,Yes
4,"Fronteiras: Journal of Social, Technological a...",NaN,2238-8869,Portuguese,Centro Universitário de Anápolis,No,CC BY-NC,No
...,...,...,...,...,...,...,...,...
193,Revista Analisando em Ciência da Informação,NaN,2317-9708,Portuguese,Universidade Estadual da Paraíba,No,CC BY-NC,No
194,"Rainbow: Journal of Literature, Linguistics an...",2252-6323,2721-4540,English,Universitas Negeri Semarang,No,CC BY,No
195,Jurnal Bidang Pendidikan Dasar,2549-0117,2549-0125,Indonesian,Universitas Kanjuruhan Malang,No,CC BY-SA,Yes
196,Science Diliman,0115-7809,2012-0818,English,University of the Philippines,No,Publisher's own license,No


In [ ]:
import json

path = 'test_data/scimago.json'
with open(path, 'r') as f:
    file = json.load(f)

if isinstance(file, dict):
    for key, value in file.items():
        print(f"Chiave: {key}, Valore: {value}")
        break
else:
    print("L'oggetto JSON caricato non è un dizionario.")

L'oggetto JSON caricato non è un dizionario.


In [ ]:
def getIdent_Cat_dict(self, issn):
        # create df with ALL identifiers (there will be repetition) and category_id
        identity_category_dict = dict()
        with connect(self.dbPathOrUrl) as con:
            query = """
            SELECT identifiers, category_id
            FROM info
            WHERE category_id IS NOT NULL AND category_id != '';
            """
            df = pd.read_sql_query(query, con)
        for index, row in df.iterrows():
                identifier = row['identifiers']
                # in the dictionary there will be no more repetition
                if identifier not in identity_category_dict:
                    identity_category_dict[identifier] = []
                identity_category_dict[identifier].append(row['category_id'])
        return identity_category_dict

def getIdentities_with_category(self):
        """
        returns a dictionary (value always = []) with listed identifiers having at least one category
            if dictionary already exist skip return it directly
        """
        identity_category_dict = dict()
        with connect(self.dbPathOrUrl) as con:
            query = """
            SELECT DISTINCT identifiers
            FROM info
            WHERE category_id IS NOT NULL AND category_id != '';
            """
            df = pd.read_sql_query(query, con)
        for index, row in df.iterrows():
                identifier = row['identifiers']
                if identifier not in identity_category_dict:
                    identity_category_dict[identifier] = []
        return identity_category_dict

def getCategory(self, issn):
        """
        return a list of categories associated with given issn
        """
        category_list = list()
        # print(len(self.categoryQuery))
        if len(self.categoryQuery) > 0:
            # category_df = pd.DataFrame()
            all_category_dfs = list()
            for handler in self.categoryQuery:
                new_category_df = handler.getIdentity_and_category()
                all_category_dfs.append(new_category_df)
            
            if all_category_dfs:
                # print('if...')
                category_df = pd.concat(all_category_dfs, ignore_index=True)
            else:
                # print('else...')
                category_df = pd.DataFrame()
        category_list = category_df.loc[category_df['identifiers'] == issn, 'category_id'].tolist()
        print(category_list)
        return category_list

def getAreasAssignedToCategories(self, categories=None):
        with connect(self.dbPathOrUrl) as con:
            if categories:
                # Converti l'insieme in una lista (o tupla) per l'uso con IN e i placeholder
                categories_list = list(categories)
                placeholders = ', '.join('?' * len(categories_list))
                query = f"SELECT DISTINCT area FROM info WHERE category_id IN ({placeholders})"
                df = pd.read_sql_query(query, con, params=categories_list)
            else:
                query = "SELECT DISTINCT area FROM info"
                df = pd.read_sql_query(query, con)
            df_sorted = df.sort_values(by='area')
        return df_sorted


def getAllJournals(self) -> list[Journal]:
        # create list for identity with category or area
        if len(self.categoryQuery) > 0:
            for handler in self.categoryQuery:
                identity_with_category = handler.getIdent_Cat_dict()
                identity_with_area = handler.getIdentities_with_area()
        print('id_with_cat:...', identity_with_category)
        # print('la mia lista', identity_with_category)
        journal_list = list()
        if len(self.journalQuery) > 0:
            all_journal_dfs = []
            for handler in self.journalQuery:
                print(handler)
                new_journal_df = handler.getAllJournals()
                all_journal_dfs.append(new_journal_df)

            if all_journal_dfs:
                # concatenate all journal df in the list
                journal_df = pd.concat(all_journal_dfs, ignore_index=True)
                # print(journal_df.info())
                # remove duplicates based on 'journal' name
                journal_df.drop_duplicates(subset=['journal'], keep='first', inplace=True, ignore_index=True)
                
            else:
                # if all_journal_dfs == False: return empty df
                journal_df = pd.DataFrame() 
        # print(journal_df.info())
        # convert df into list of Python Objects
        for index, row in journal_df.iterrows():
            # print('index number: ', index)
            print(row['issn'])
            # if row['issn'] in identity_with_category:
                # print('fatta la lista categorie:', self.getCategory(row['issn']))
            # else:
                # print('fatta la lista categorie:', [])
            journal = Journal(
                id=[row['journal']],  
                title=row['title'],
                languages=[row['languages']] if pd.notna(row['languages']) else [],
                publisher=row['publisher'] if pd.notna(row['publisher']) else None,
                seal=row['seal'] if pd.notna(row['seal']) and str(row['seal']).lower() == 'yes' else False,
                license=row['license'] if pd.notna(row['license']) else None,
                apc=row['apc'] if pd.notna(row['apc']) and str(row['apc']).lower() == 'yes' else False,
                hasCategory= identity_with_category[row['issn']] if row['issn'] in identity_with_category else [],
                hasArea= self.getArea(row['issn']) if row['issn'] in identity_with_area else []
            ) 
            journal_list.append(journal)

        return journal_list # journal_list, len(journal_list)